In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cmaps
import cartopy.feature as cfeature
from  scipy  import stats
import matplotlib as mpl
import cartopy.io.shapereader as shpreader
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
from eofs.standard import Eof
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
import warnings
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
file1 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1979-2024.nc")
precip = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),:,:].groupby('time.year').mean('time').rename({'year':'time'})*1000 # unit converts to mm/day
precip_ism = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),28:10,70:100].groupby('time.year').mean('time').rename({'year':'time'})
precip_index = precip_ism.mean(('longitude','latitude'))*1000
precip_index = (precip_index - np.mean(precip_index,axis=0))/np.std(precip_index)
lon = precip.longitude
lat = precip.latitude
data = precip.values
precip_mean = np.nanmean(data,0)
precip_std = np.std(data,axis=0)

In [ ]:
file2 = xr.open_dataset("/public/home/songqh/my_data/NCEP1/monthly/precipitation_monthly/GPM.precipitation.monthly.nc")
gpm_precip = file2['precipitation'].loc[file2.time.dt.year.isin(np.arange(1998,2025,1))&file2.time.dt.month.isin([6,7,8]),:,:].groupby('time.year').mean('time').rename({'year':'time'}) # unit converts to mm/day
gpm_precip_ism= file2['precipitation'].loc[file2.time.dt.year.isin(np.arange(1998,2025,1))&file2.time.dt.month.isin([6,7,8]),70:100,10:28].groupby('time.year').mean('time').rename({'year':'time'})
gpm_precip_index = gpm_precip_ism.values.mean((1,2))*1000
gpm_precip_index = (gpm_precip_index - np.mean(gpm_precip_index,axis=0))/np.std(gpm_precip_index)

In [ ]:
file0 = xr.open_dataset("/public/home/songqh/my_data/ice_cover/HadISST_ice.nc")
file0.coords['longitude'] = np.mod(file0['longitude'], 360)
file0 = file0.reindex({ 'longitude' : np.sort(file0['longitude'])})
sic = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1))
                        & file0.time.dt.month.isin([6,7,8]),-30:-90].groupby('time.year').mean('time')
sic_mean = sic.mean('year')
sic_ano = sic - sic_mean
lon1 = file0['longitude'].loc[120:350]
lat1 = file0['latitude'].loc[-40:-90]
sic_zone = sic_ano.loc[:,-40:-90,120:350].values

In [ ]:
lat1 = np.array(lat1)
coslat = np.cos(np.deg2rad(lat1))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(sic_zone, weights=wgts)
eof =solver.eofsAsCorrelation(neofs=3)
pc = solver.pcs(npcs=3, pcscaling=1)
var = solver.varianceFraction()
pc1 = pc[:,0]
pc2 = pc[:,1]
eof1 = eof[0,:,:]
eof2 = eof[1,:,:]
print("var:",var[:3])

In [ ]:
r1,p1 = stats.pearsonr(precip_index,pc2[:])
print("ERA5_rainfall correlation:",r1,"p:",p1)

In [ ]:
r2,p2 = stats.pearsonr(gpm_precip_index,pc2[19:])
print("GPM_rainfall correlation:",r2,"p:",p2)

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
ncl_cmap = cm.get_cmap(cmaps.CBR_wet, 256)  
cmap_colors = ncl_cmap(np.linspace(0, 1, 256))  
cmap_colors = cmap_colors[30:]
modified_CBR_wet = LinearSegmentedColormap.from_list("modified_CBR_wet", cmap_colors)

In [ ]:
fig = plt.figure(figsize=(16,12.5))
ax1 = fig.add_axes([0.05,0.585,0.3,0.3],projection = ccrs.PlateCarree(central_longitude=120))
ax1.draw_map([60,110,0,40],lonspec=10,latspec=10,res=110)
ax1.set_xticks([70, 90, 110], crs=ccrs.PlateCarree())
ax1.set_xticklabels(['70°E', '90°E', '110°E'])
ax1.set_title("JJA rainfall average",fontsize=20,loc='center')
ax1.set_title("a",fontsize=24,weight='bold',loc='left')
shp_path = "/public/home/songqh/Documents/shape/tibet/DBATP_Polygon.shp"  
reader = shpreader.Reader(shp_path)
for record in reader.records():
    geometry = record.geometry
    ax1.add_geometries([geometry], crs=ccrs.PlateCarree(), facecolor='none', zorder=10,edgecolor='purple')
cf1 = ax1.contourf(lon[:],lat[:],precip_mean[:,:], zorder=2,levels=np.linspace(0,25,11),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=modified_CBR_wet)
rect = plt.Rectangle((70, 10), 30, 18, edgecolor='#BF1D2D',zorder=10,
                     facecolor='None',linewidth=1.5,transform=ccrs.PlateCarree())
ax1.add_patch(rect)
pos = fig.add_axes([0.36,0.63,0.01,0.2])
cb = fig.colorbar(cf1,orientation='vertical',cax=pos)
cb.ax.set_colorbar(minor='off')

map_proj = ccrs.LambertConformal(
    central_longitude=230, standard_parallels=(-60, -50))
ax2 = fig.add_axes([0.421,0.585,0.5,0.3],projection =map_proj)
ax2.set_extent([120, 340, -90, -55], crs=ccrs.PlateCarree())
ax2.spines['geo'].set_linewidth(1)
ax2.spines['geo'].set_zorder(4)
ax2.spines['geo'].set_edgecolor('black')
cf1 = ax2.contourf(lon1,lat1,-eof[1,:,:], zorder=1,levels=np.linspace(-1,1,21),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_cmocean_balance_r)
ax2.add_feature(cfeature.COASTLINE.with_scale('110m'),color='k',linewidth=1,zorder=4)
ax2.add_feature(cfeature.LAND.with_scale('110m'),color='lightgray',zorder=3)
ax2.set_title("JJA SIC EOF2 (var=15%)",fontsize=20,loc='center')
ax2.set_title("c",fontsize=24,weight='bold',loc='left')
gl = ax2.gridlines(draw_labels=False, linestyle='--', color='gray', alpha=0.7, linewidth=0.5,zorder=4)
pos = fig.add_axes([0.918,0.63,0.01,0.2])
cb = fig.colorbar(cf1,orientation='vertical',cax=pos)
cb.ax.set_colorbar(minor='off')

ax3 = fig.add_axes([0.05,0.22,0.3,0.3],projection = ccrs.PlateCarree(central_longitude=120))
ax3.draw_map([60,110,0,40],lonspec=10,latspec=10,res=110)
ax3.set_xticks([70, 90, 110], crs=ccrs.PlateCarree())
ax3.set_xticklabels(['70°E', '90°E', '110°E'])
ax3.set_title("JJA rainfall S.D.",fontsize=20,loc='center')
ax3.set_title("b",fontsize=24,weight='bold',loc='left')
reader = shpreader.Reader(shp_path)
for record in reader.records():
    geometry = record.geometry
    ax3.add_geometries([geometry], crs=ccrs.PlateCarree(), facecolor='none', zorder=10,edgecolor='purple')
cf1 = ax3.contourf(lon[:],lat[:],precip_std[:,:], zorder=2,levels=np.linspace(0,4,11),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=modified_CBR_wet)
rect = plt.Rectangle((70, 10), 30, 18, edgecolor='#BF1D2D',zorder=10,
                     facecolor='None',linewidth=1.5,transform=ccrs.PlateCarree())
ax3.add_patch(rect)
pos = fig.add_axes([0.36,0.27,0.01,0.2])
cb = fig.colorbar(cf1,orientation='vertical',cax=pos)
cb.ax.set_colorbar(minor='off')

ax4 = fig.add_axes([0.435,0.22,0.471,0.3])
ax4.plot(np.arange(1979,2025),precip_index,c="#d76364",lw=2.5,markersize=8,label='ERA5-rainfall',zorder=1)
ax4.plot(np.arange(1979,2025),-pc2,zorder=2,color='#2F7FC1',lw=2.5,markersize=8,label='SIC-PC2')
ax4.plot(np.arange(1998,2025),gpm_precip_index,zorder=0,linestyle='--',c='k',lw=2.5,markersize=8,label='GPM-rainfall')
ax4.set_detail()
ax4.axhline(0,c='k',lw=0.5,linestyle='--')
ax4.set_ylim(-3,3.4)
ax4.set_xlim(1978,2025)
ax4.set_title("JJA rainfall & SIC-PC2",fontsize=20,loc='center')
ax4.set_title("d",fontsize=24,weight='bold',loc='left')
ax4.text(1990,-2.5,"r(ERA5-rainfall&PC2)=0.50 (P<0.01)",fontsize=18)
ax4.legend(frameon=False,fontsize=18,ncol=2,handlelength=1.15,handletextpad=0.3,columnspacing=1.5,labelspacing=0.5,loc='upper left')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/response_figures/figure-1.jpeg",dpi=600, bbox_inches='tight')